### Exploratory Data Analysis using Python

Importing necessary libraries:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("paper", font_scale=1.0) 

%matplotlib inline

In [ ]:
# Set Options for display
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 100
pd.options.display.float_format = '{:.2f}'.format

#Filter Warnings
import warnings
warnings.filterwarnings('ignore')

Load the dataset:

In [ ]:
# to check your working directory
%pwd

# to change your working directory, use
# %cd

In [ ]:
df = pd.read_csv('/Users/radgerald/Documents/EDA for Lexmark/datasets/Ad-data.csv')

Describe the data:

In [ ]:
df.head()

In [ ]:
# check if the data is loaded correctly
df.tail()

In [ ]:
# check the dimensions
df.shape

In [ ]:
# check the features
df.info()

In [ ]:
# check the column names
df.columns

## Verify Data Quality

Look out for the following:
- Unexpected missing values
- Incorrect or unexpected data types and formats
- Duplicates
- Unexpected dimensions (i.e. missing rows or columns)
- Incorrect spelling
- Unexpected outliers or anomalous values
- Inconsistent or incorrect units of measurement 


In [ ]:
# check for unexpected missing values
total = df.isnull().sum().sort_values(ascending=False)
total

In [ ]:
### Assume blank link_clicks can be represented by 0, drop the column
df['link_clicks'].fillna(0, inplace=True)

In [ ]:
total = df.isnull().sum().sort_values(ascending=False)
total

In [ ]:
# check for incorrect or unexpected data type and format
df.dtypes

In [ ]:
# Convert Date to datetime format
df['Date']= pd.to_datetime(df['Date'])

In [ ]:
# check for duplicates
df.duplicated()

In [ ]:
df.duplicated().value_counts()

In [ ]:
# Assume upon checking with data owner that duplicates are not expected and should be dropped

In [ ]:
# handling duplicates
df.drop_duplicates(inplace=True)

In [ ]:
df.duplicated().value_counts()

In [ ]:
# check categorical columns
# split the dataset into categorical and numerical features

# categorical features Only
df_cat = df.select_dtypes(include=['object'])

# numerical features Only
df_num = df.select_dtypes(include=['int64','float64'])

In [ ]:
# check for misspellings and mixed cases for categorical data

# describe the Categorical Features. Check if Number of Unique Values is as expected. 
df.describe(include=['object'])

In [ ]:
# other Methods to Check: Value Counts
df_cat.age.value_counts()

In [ ]:
# other Methods to Check: Unique
df_cat.campaign_platform.unique()

In [ ]:
# other Methods to Check: Number of Unique
df_cat.campaign_type.nunique()

In [ ]:
# for loop to go over all the features and print out the unique values for each feature.

for cat_col in df_cat.columns:
    print (df[cat_col].value_counts())
    print ("\n---------")

In [ ]:
# extract data features

df['Year'] = df.Date.dt.year
df['Month'] = df.Date.dt.month
df['Day']=df.Date.dt.day
df['Week'] = df.Date.dt.week

In [ ]:
df.groupby('Month')['clicks'].max()

In [ ]:
# check numneric columns

# check skewness

# check the Number of Numerical Features
df_num.shape

In [ ]:
# describe the Numerical Features. Check if the Mean, Min, and Max values are as expected.
df_num.describe()

In [ ]:
# All features can be plotted on a histogram
df.clicks.hist()

In [ ]:
sns.distplot(df['clicks'])

In [ ]:
# box plots

sns.boxplot(df['clicks'])

What can you obseve about the target variable Sale Price?
- How is it distributed?
- Is the data skewed?
- Are there outliers?

In [ ]:
# handling skewness

#sns.kdeplot(data = df, x = 'clicks')
sns.kdeplot(df.clicks)

In [ ]:
# check conversions

df['clicks_log'] = np.log(1+df['clicks'])
sns.kdeplot(data = df['clicks_log'])

In [ ]:
df['clicks_sqrt'] = np.sqrt(df['clicks'])
sns.kdeplot(data = df['clicks_sqrt'])

In [ ]:
print('Original:' , df['clicks'].skew())
print('Log Transform:' , df['clicks_log'].skew())
print('Square Root Transform:' , df['clicks_sqrt'].skew())

In [ ]:
df['spends_log'] = np.log(1+df['spends'])
df['spends_sqrt'] = np.sqrt(df['spends'])
print('Original:' , df['spends'].skew())
print('Log Transform:' , df['spends_log'].skew())
print('Square Root Transform:' , df['spends_sqrt'].skew())

In [ ]:
df['impressions_log'] = np.log(1+df['impressions'])
df['impressions_sqrt'] = np.sqrt(df['impressions'])
print('Original:' , df['impressions'].skew())
print('Log Transform:' , df['impressions_log'].skew())
print('Square Root Transform:' , df['impressions_sqrt'].skew())

In [ ]:
df['link_clicks_log'] = np.log(1+df['link_clicks'])
df['link_clicks_sqrt'] = np.sqrt(df['link_clicks'])
print('Original:' , df['link_clicks'].skew())
print('Log Transform:' , df['link_clicks_log'].skew())
print('Square Root Transform:' , df['link_clicks_sqrt'].skew())

In [ ]:
df.drop(['clicks_sqrt','spends_sqrt','impressions_sqrt', 'link_clicks_sqrt'], axis = 1, inplace = True)

In [ ]:
df.clicks_log.hist()

In [ ]:
sns.boxplot(data = df['clicks_log'],orient='h')

In [ ]:
# handling unexpected outliers

# can remove outliers using the IQR method where any value below Q1-1.5*IQR or above Q3+1.5*IQR is an Outlier

def remove_Outliers(df,col_name):
    print("Orig DF Size:"+ str(df.shape) )
    Q1 = np.quantile(df[col_name],0.25)

    Q3 = np.quantile(df[col_name],0.75)

    IQR = Q3 - Q1

    lower_limit = Q1 - (1.5*IQR)
    upper_limit = Q3 + (1.5*IQR)

    print("Lower Limit: %.2f" % lower_limit)
    print("Upper Limit: %.2f" % upper_limit)
    
    df_new = df[(df[col_name] > lower_limit) & (df[col_name] < upper_limit)]
    print("New DF Size:"+ str(df_new.shape) )
    return df_new


In [ ]:
df.shape

In [ ]:
df_temp = remove_Outliers(df, 'clicks_log')
df_temp.shape

In [ ]:
df_temp.clicks_log.hist()

In [ ]:
# make outlier removal permanent
df = df_temp

In [ ]:
# Categorical Variables

# for categorical variables, you can use a countplot

sns.countplot(df.campaign_platform)

Multivariate Plots to Understand Relatonships between Features, Test Hypothesis and Check Assumptions

In [ ]:
# correlation matrix
corrmat = df.corr()

In [ ]:
corrmat

In [ ]:
sns.heatmap(corrmat,  cmap="vlag", center = 0,  vmax=1, square=True, linewidths=.5)

In [ ]:
# Feature sorted by correlation to Clicks_log, from positive to negative
corr = corrmat.sort_values('clicks_log', ascending=False)

sns.barplot(corr.clicks_log[1:], corr.index[1:], orient='h')

plt.show()

Quantitative vs Quantitative

In [ ]:
sns.scatterplot(x='spends', y='clicks', data = df)

In [ ]:
sns.scatterplot(x='spends_log', y='clicks_log', data = df)

In [ ]:
sns.regplot(x='spends', y='clicks', data = df)

In [ ]:
sns.jointplot(x='spends_log', y='clicks_log', data=df)

In [ ]:
# Impressions vs Spends
sns.jointplot(x='spends_log', y='impressions_log', data=df, kind='reg')

In [ ]:
df_metrics = df[['campaign_platform','spends','spends_log', 'clicks', 'clicks_log', 'impressions', 'impressions_log','link_clicks','link_clicks_log']]

#Original Code
#sns.pairplot(df_metrics, hue='campaign_platform', kind='reg')

#Alternative Code
sns.pairplot(df_metrics, hue='campaign_platform', diag_kws={'bw': 1})

Violin Plots

In [ ]:
# Violinplots on SalePrice given Functional feature
sns.violinplot(data=df,x="campaign_type", y="clicks_log")

In [ ]:
# Violinplots on SalePrice given Functional feature
sns.stripplot(data=df,x="subchannel", y="clicks")

In [ ]:
sns.catplot(data=df,x="age", y="clicks_log", hue = "device", kind="point")

In [ ]:
sns.catplot(data=df,x="age", y="clicks_log", hue = "subchannel", kind="point")

In [ ]:
sns.barplot(data=df, x='Month', y='clicks')

In [ ]:
sns.boxplot(data=df, x='Month', y='clicks_log')

Data Preparation - Category to Numeric

Convert ordinal features to numeric:

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
df.age.values

In [ ]:
sns.countplot(data=df, x=df.age)

In [ ]:
df.age.value_counts()

In [ ]:
#Using Label Encode
le = LabelEncoder()
df["age_le"] = le.fit_transform(df['age'])

In [ ]:
df.age_le

In [ ]:
age_vc = df.age.value_counts().sort_values(ascending=False)
age_le_vc = df.age_le.value_counts().sort_values(ascending=False)
pd.DataFrame([age_vc.index.values,age_le_vc.index.values,age_vc.values], index=['age','age_le','values']).T

In [ ]:
# Using substitution
df['age_ord'] = df['age']
df = df.replace({'age_ord':{'18-24':1,'25-34':2,'35-44':3, 'Undetermined': 4, '45-54':5, '55-64': 6,'65 or more':7}})

In [ ]:
age_vc = df.age.value_counts().sort_values(ascending=False)
age_ord_vc = df.age_ord.value_counts().sort_values(ascending=False)
pd.DataFrame([age_vc.index.values,age_ord_vc.index.values,age_vc.values], index=['age','age_ord','values']).T

In [ ]:
#Drop age column and keep age_ord

df.drop(['age', 'age_le'],axis = 1, inplace = True)

Convert nominal features to numeric

In [ ]:
df.dtypes

In [ ]:
df_cat = df.select_dtypes(include='object')

df_cat_dummies = pd.get_dummies(df_cat)

In [ ]:
df_cat_dummies.head()

In [ ]:
#join the encoded columns to the original dataframe
df = df.join(df_cat_dummies)

In [ ]:
df.head()

In [ ]:
#drop original columns
df.drop(columns = df_cat.columns, axis = 1, inplace = True)

In [ ]:
df.head()

In [ ]:
#Check correlations of all columns
corrmat = df.corr()
plt.figure(figsize=(10,10))
sns.heatmap(corrmat,  cmap="vlag", center = 0,  vmax=.9, square=True, linewidths=.5)

Data Preparation - Scaling

In [ ]:
df.shape

In [ ]:
df.clicks.describe()

In [ ]:
df.describe()

In [ ]:
#Separate Features and Target
df_x = df.drop(["clicks","clicks_log"], axis=1)
y = df[["clicks","clicks_log"]]

In [ ]:
df_x.drop("Date", axis=1,inplace = True)

In [ ]:
#Import the MinMax Scaler
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

In [ ]:
#NOTE: Only scale the predictor variables, NOT the target variable

#Instantiate the Scaler
scaler = MinMaxScaler()

#Fit to the data set
scaler.fit(df_x)

#Apply to the data set
scaled_data = scaler.transform(df_x)


#Optional:
#Convert to DataFrame for viewing
df_minmax = pd.DataFrame(scaled_data, columns=df_x.columns, index=df_x.index)

In [ ]:
df_minmax.describe()

In [ ]:
#combine scaled data with target columns
df_prep = df_minmax.join(y)

df_prep.head()

In [ ]:
#Save as a csv
df_prep.to_csv('/Users/radgerald/Documents/EDA for Lexmark/datasets/Ad-data_prep.csv')